# Notebook for training/testing MLP network

We train a set of `MLP` networks on different datasets (**for different n central features, L - fixed**)

In [ ]:
import os
import random
import numpy as np

from os.path import join as p_join

In [ ]:
###############################
##### IMPORT DL Depensies #####
###############################
import torch

def seed_all(seed=42):

    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print("[ Using Seed : ", seed, " ]")

####################################
#####   SEED ALL EXPERIMENTS   #####
####################################
seed_all()    

L_PARAMETR = 10.6                         # Set L parametr to create dataset
#################################
####### GLOBAL CONFIG ###########
#################################
CONFIG = {'ephs': 100,
          'train_batch_size': 1024,
          'test_batch_size': 64,
          'cpu_workers': 2,
          'save_ckpts': False,
          'ckpt_save_folder': f'mlp_ckpts_L_{L_PARAMETR}'}

In [ ]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utils import utils
from utils import dl_models

In [ ]:
data_path = p_join('..', '..', 'data')
FOLDERS = [p_join(os.path.abspath(data_path), item)
           for item in os.listdir(data_path) if f'L={L_PARAMETR}' in item]

### Run training

In [ ]:
results = {}
models, test_loaders = {}, {}
for n_feat in [16, 17, 18, 19, 20]:
    CONFIG['ckpt_save_folder'] = f'mlp_ckpts_L_{L_PARAMETR}_feat_{n_feat}'
    CONFIG['experiment_name'] = f'L={L_PARAMETR}, N={n_feat}'
    print(f"Starting {CONFIG['experiment_name']} ...")
    ################################
    ####    Create dataset      ####
    ################################
    seed_all()
    X, Y = utils.create_dataset(FOLDERS, feature_nums=n_feat)
    X = utils.normalize_data(X)
    train_dataloader, test_dataloader = utils.create_dataloaders(
                                            X, Y,
                                            cpu_workers=CONFIG['cpu_workers'],
                                            train_bs=CONFIG['train_batch_size'],
                                            test_bs=CONFIG['test_batch_size']
                                        )
    
    ################################
    ####    Run training        ####
    ################################
    net = dl_models.MlpNetLight(in_features=n_feat)
    utils.count_params(net)
    optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 5e-3))
    score = utils.run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)

    models[CONFIG['experiment_name']] = net
    test_loaders[CONFIG['experiment_name']] = test_dataloader
    results[CONFIG['experiment_name']] = {'Accuracy': score}

In [ ]:
results